In [5]:
import sagemaker
sess = sagemaker.Session()
role = "arn:aws:iam::842977422697:role/SageMaker_Cinnamon_role"
account = sess.boto_session.client('sts').get_caller_identity()['Account']
region = sess.boto_session.region_name

# ECR repository
repo_name = 'sagemaker-hades' 
# local + ECR image tag
image_tag = 'hades01'

%env dockerfile Dockerfile
%env account {account}
%env region {region}
%env repo_name {repo_name}
%env image_tag {image_tag}

env: dockerfile=Dockerfile
env: account=842977422697
env: region=us-west-2
env: repo_name=sagemaker-hades
env: image_tag=hades01


In [18]:
%%bash

aws ecr describe-repositories --repository-names sagemaker-hades > /dev/null 2>&1
if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name sagemaker-hades > /dev/null
fi

In [20]:
%%bash

aws ecr get-login-password --region us-west-2 | docker login --username AWS --password-stdin 842977422697.dkr.ecr.us-west-2.amazonaws.com

Login Succeeded


WARNING! Your password will be stored unencrypted in /home/hades/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store



In [12]:
!docker build -t $image_tag -f $dockerfile .
!docker tag $image_tag $account.dkr.ecr.$region.amazonaws.com/$repo_name:$image_tag

'docker' is not recognized as an internal or external command,
operable program or batch file.
'docker' is not recognized as an internal or external command,
operable program or batch file.


In [ ]:
!docker push $account.dkr.ecr.$region.amazonaws.com/$repo_name:$image_tag